<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/ga_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch1.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import os
import glob
for i in range(5,28):
    os.remove( '/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-'+str(i)+'.nii')
    os.remove( '/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-'+str(i)+'.nii')

In [ ]:
!pip install medpy

     |████████████████████████████████| 153kB 11.6MB/s 
     |████████████████████████████████| 47.4MB 98kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754459 sha256=349b481085df830ace6af3ef4a330c4927526560f716e9ffe0ceecca28422483
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [ ]:
import shutil
for path in glob.glob('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-*'):
  shutil.move(path,'/content/data/myTrainingData')
for path in glob.glob('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-*'):
  shutil.move(path,'/content/data/myTrainingData')

In [ ]:

from medpy.io import load, save
import os
import os.path
import numpy as np


def proprecessing(image_path, save_folder):

    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)
    filelist = os.listdir(image_path)
    imagelist = [item for item in filelist if 'volume' in item]
    seglist = [item for item in filelist if 'volume' in item]
    for id in range(5):
        img, img_header = load('/content/data/myTrainingData/volume-'+str(id)+'.nii')
        seg, img_header = load('/content/data/myTrainingData/segmentation-'+str(id)+'.nii')
        img[img < -200] = -200
        img[img > 250] = 250
        img = np.array(img, dtype='float32')
        print ("Saving image "+str(id))
        tot=np.concatenate([img,seg],-1)
        np.save( "/content/data/" + save_folder + str(id),tot)

def generate_livertxt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'LiverPixels'):
        os.mkdir("/content/data/"+save_folder+'LiverPixels')

    for i in range(0,5):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open('/content/data/' +save_folder+'/LiverPixels/liver_' + str(i) + '.txt', 'w')
        index = np.where(livertumor==1)
        x = index[0]
        y = index[1]
        z = index[2]
        np.savetxt(f, np.c_[x,y,z], fmt="%d")
	
        f.write("\n")
        f.close()

def generate_tumortxt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'TumorPixels'):
        os.mkdir("/content/data/"+save_folder+'TumorPixels')

    for i in range(0,5):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open("/content/data/"+save_folder+"/TumorPixels/tumor_"+str(i)+'.txt','w')
        index = np.where(livertumor==2)

        x = index[0]
        y = index[1]
        z = index[2]

        np.savetxt(f,np.c_[x,y,z],fmt="%d")

        f.write("\n")
        f.close()

def generate_txt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'LiverBox'):
        os.mkdir("/content/data/"+save_folder+'LiverBox')
    for i in range(0,5):
        values = np.loadtxt('/content/data/myTrainingDataTxt/LiverPixels/liver_' + str(i) + '.txt', delimiter=' ', usecols=[0, 1, 2])
        a = np.min(values, axis=0)
        b = np.max(values, axis=0)
        box = np.append(a,b, axis=0)
        np.savetxt('/content/data/myTrainingDataTxt/LiverBox/box_'+str(i)+'.txt', box,fmt='%d')


proprecessing(image_path='/content/data/myTrainingData/', save_folder='myTrainingData/')
print ("Generate liver txt ")
generate_livertxt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate tumor txt")
generate_tumortxt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate liver box ")
generate_txt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')

Saving image 0
Saving image 1
Saving image 2
Saving image 3
Saving image 4


In [ ]:

def load_seq_crop_data_masktumor_try(Parameter_List):
    img = Parameter_List[0]
    tumor = Parameter_List[1]
    lines = Parameter_List[2]
    numid = Parameter_List[3]
    minindex = Parameter_List[4]
    maxindex = Parameter_List[5]
    #  randomly scale
    scale = np.random.uniform(0.8,1.2)
    deps = int(args.input_size * scale)
    rows = int(args.input_size * scale)
    cols = args.input_cols

    sed = np.random.randint(1,numid)
    cen = lines[sed-1]
    cen = np.fromstring(cen, dtype=int, sep=' ')
    # print (cen)
    a = min(max(minindex[0] + deps/2, cen[0]), maxindex[0]- deps/2-1)
    b = min(max(minindex[1] + rows/2, cen[1]), maxindex[1]- rows/2-1)
    c = min(max(minindex[2] + cols/2, cen[2]), maxindex[2]- cols/2-1)

    #change
    #TypeError: slice indices must be integers or None or have an __index__ method

    # try:
    if a < deps / 2:
      a1=int((img.shape[0]/2)-(deps/2))
      a2=int((img.shape[0]/2)+(deps/2))
    else:
      a1=int(a-deps/2)
      a2=int(a+deps/2)
    if b < rows / 2:
      b1=int((img.shape[0]/2)-(rows/2))
      b2=int((img.shape[0]/2)+(rows/2))
    else:
      b1=int(b-rows/2)
      b2=int(b+rows/2)
    if c < cols / 2:
      c1=int((img.shape[0]/2)-(cols/2))
      c2=int((img.shape[0]/2)+(cols/2))
    else:
      c1=int(c-cols/2)
      c2=int(c+cols/2)

    cropp_img = img[a1:a2, b1:b2,c1: c2+1].copy()

  #
    cropp_tumor = resize(cropp_tumor, (args.input_size,args.input_size, args.input_cols), order=0, mode='edge', cval=0, clip=True, preserve_range=True)
    cropp_img   = resize(cropp_img, (args.input_size,args.input_size, args.input_cols), order=3, mode='constant', cval=0, clip=True, preserve_range=True)
    return cropp_img, cropp_tumor
    
def generate_arrays_from_file(batch_size, trainidx, img_list, tumor_list, tumorlines, liverlines, tumoridx, liveridx, minindex_list, maxindex_list):
        X = np.zeros((batch_size, args.input_size, args.input_size, args.input_cols,1), dtype='float32')
        Parameter_List = []
        for idx in range(batch_size):
            count = np.random.choice(trainidx)
            img = img_list[count]
            tumor = tumor_list[count]
            minindex = minindex_list[count]
            maxindex = maxindex_list[count]
            num = np.random.randint(0,6)
            if num < 3 or (count in liverlist):
                lines = liverlines[count]
                numid = liveridx[count]
            else:
                lines = tumorlines[count]
                numid = tumoridx[count]
            Parameter_List.append([img, tumor, lines, numid, minindex, maxindex])
        pool = ThreadPool(thread_num)
        result_list = pool.map(load_seq_crop_data_masktumor_try, Parameter_List)
        pool.close()
        pool.join()
        for idx in range(len(result_list)):
            X[idx, :, :, :, 0] = result_list[idx][0]
            Y[idx, :, :, :, 0] = result_list[idx][1]
        if np.sum(Y==0)==0:
            continue
        if np.sum(Y==1)==0:
            continue
        if np.sum(Y==2)==0:
            continue
        yield (X,Y)

In [ ]:
for idx in range(5):
        img, img_header = load('/content/data/myTrainingData/volume-'+str(idx)+'.nii')
        img_list.append(img)

        maxmin = np.loadtxt('/content/data/myTrainingDataTxt/LiverBox/box_'+str(idx)+'.txt')
        minindex = maxmin[0:3]
        maxindex = maxmin[3:6]
        minindex = np.array(minindex, dtype='int')
        maxindex = np.array(maxindex, dtype='int')
        minindex[0] = max(minindex[0]-3, 0)
        minindex[1] = max(minindex[1]-3, 0)
        minindex[2] = max(minindex[2]-3, 0)
        maxindex[0] = min(img.shape[0], maxindex[0]+3)
        maxindex[1] = min(img.shape[1], maxindex[1]+3)
        maxindex[2] = min(img.shape[2], maxindex[2]+3)
        minindex_list.append(minindex)
        maxindex_list.append(maxindex)

        f1 = open('/content/data/myTrainingDataTxt/TumorPixels/tumor_'+str(idx)+'.txt')
        tumorline = f1.readlines()
        tumorlines.append(tumorline)
        tumoridx.append(len(tumorline))
        f1.close()

        f2 = open('/content/data/myTrainingDataTxt/LiverPixels/liver_'+str(idx)+'.txt')
        liverline = f2.readlines()
        liverlines.append(liverline)
        liveridx.append(len(liverline))
        f2.close()
generate_arrays_from_file(1, trainidx, img_list, tumorlines, liverlines,
                                                  tumoridx, liveridx, minindex_list, maxindex_list)